In [34]:
import os
import shutil

# 🏷 Set base folder path and list of subfolders to delete
base_folder = "/content/classified_emojis"
subfolders_to_delete = [
    "Animals",
    "Art & Creativity",
    "classified_emojis",
    "Birds & Insects",
    "Birds",
    "Building & Places",
    "Celebration & Events",
    "Emoji Keycaps & Digits",
    "Expressions & Faces",
    "Family & Relationships",
    "Flags",
    "Food & Drinks",
    "Hand Gestures",
    "Kitchen & Household",
    "Love & Emotions",
    "Mail & Communication Symbols",
    "Medical & Health",
    "Modifiers (Tones) & Extras",
    "Money & Finance",
    "Music & Instruments",
    "Nature & Plants",
    "People & Body",
    "Signs & Symbols",
    "Sports & Games",
    "Technology & Devices",
    "Time & Clocks",
    "Transport & Travel",
    "Weather & Sky",
    "Body Parts",
    "Buildings & Places",
    "Modifiers Gondi & Extras",
    "Zodiac & Astrology"
]  # Add your subfolder names here

for subfolder in subfolders_to_delete:
    subfolder_path = os.path.join(base_folder, subfolder)

    if os.path.exists(subfolder_path):
        if os.path.isdir(subfolder_path):
            subfolder_contents = os.listdir(subfolder_path)
            if len(subfolder_contents) > 0:
                print(f"Subfolder {subfolder_path} contains {len(subfolder_contents)} items. Moving contents...")
                for item in subfolder_contents:
                    src_path = os.path.join(subfolder_path, item)
                    dst_path = os.path.join(base_folder, item)

                    if os.path.exists(dst_path):
                        print(f"Skipping {dst_path}, already exists.")
                        continue

                    shutil.move(src_path, dst_path)
                    print(f"Moved {src_path} to {dst_path}")

            # Try to delete the folder
            try:
                os.rmdir(subfolder_path)
                print(f"Deleted empty subfolder: {subfolder_path}")
            except OSError as e:
                print(f"Error deleting {subfolder_path}: {e}")
                shutil.rmtree(subfolder_path)
                print(f"Force deleted subfolder: {subfolder_path}")
        else:
            print(f"{subfolder_path} is not a directory.")
    else:
        print(f"Subfolder {subfolder_path} does not exist.")

# 📊 Verify the updated structure
print(f"\nContents of {base_folder} after deletions:")
print(os.listdir(base_folder))


Subfolder /content/classified_emojis/Animals contains 94 items. Moving contents...
Moved /content/classified_emojis/Animals/U+1F40D.png to /content/classified_emojis/U+1F40D.png
Moved /content/classified_emojis/Animals/U+1F421.png to /content/classified_emojis/U+1F421.png
Moved /content/classified_emojis/Animals/U+1F9A1.png to /content/classified_emojis/U+1F9A1.png
Moved /content/classified_emojis/Animals/U+1F99B.png to /content/classified_emojis/U+1F99B.png
Moved /content/classified_emojis/Animals/U+1F995.png to /content/classified_emojis/U+1F995.png
Moved /content/classified_emojis/Animals/U+1F43B_U+200D_U+2744_U+FE0F.png to /content/classified_emojis/U+1F43B_U+200D_U+2744_U+FE0F.png
Moved /content/classified_emojis/Animals/U+1F43A.png to /content/classified_emojis/U+1F43A.png
Moved /content/classified_emojis/Animals/U+1F991.png to /content/classified_emojis/U+1F991.png
Moved /content/classified_emojis/Animals/U+1F406.png to /content/classified_emojis/U+1F406.png
Moved /content/class

In [30]:
import os

# 🏷 Set base folder path
base_folder = r"/content/classified_emojis"

# 📁 Count subfolders and images
subfolders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]
total_images = 0

print(f"Total subfolders: {len(subfolders)}")
print("Subfolders and image counts:")
for subfolder in subfolders:
    subfolder_path = os.path.join(base_folder, subfolder)
    images = [f for f in os.listdir(subfolder_path) if f.endswith('.png')]
    image_count = len(images)
    total_images += image_count
    print(f"{subfolder}: {image_count} images")

print(f"\nGrand total: {total_images} images")

Total subfolders: 0
Subfolders and image counts:

Grand total: 0 images


## **MobileNetV2**

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Set paths
train_dir = "/content/train"
val_dir = "/content/val"
img_size = 128
batch_size = 32
num_classes = 28

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_size, img_size, 3)))

# Freeze base model initially
base_model.trainable = False

# Custom head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train initial frozen model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Unfreeze last layers of base model for fine-tuning
base_model.trainable = True
fine_tune_at = 100  # Unfreeze from this layer

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Recompile with lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fine-tune
history_fine = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator
)

# Save model
model.save("mobilenetv2_finetuned.h5")


Found 4025 images belonging to 28 classes.
Found 1017 images belonging to 28 classes.


<ipython-input-2-5c56ba18663b>:44: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_tensor=Input(shape=(img_size, img_size, 3)))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_15']
Received: inputs=Tensor(shape=(None, 128, 128, 3))
  warnings.warn(msg)


126/126 ━━━━━━━━━━━━━━━━━━━━ 89s 654ms/step - accuracy: 0.5119 - loss: 1.8651 - val_accuracy: 0.7611 - val_loss: 0.8839
Epoch 2/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 142s 658ms/step - accuracy: 0.7264 - loss: 0.9600 - val_accuracy: 0.8112 - val_loss: 0.7309
Epoch 3/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 80s 634ms/step - accuracy: 0.7777 - loss: 0.7749 - val_accuracy: 0.8230 - val_loss: 0.6527
Epoch 4/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 87s 687ms/step - accuracy: 0.8079 - loss: 0.6269 - val_accuracy: 0.8407 - val_loss: 0.5948
Epoch 5/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 87s 690ms/step - accuracy: 0.8038 - loss: 0.6468 - val_accuracy: 0.8368 - val_loss: 0.5909
Epoch 6/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 86s 685ms/step - accuracy: 0.8395 - loss: 0.5064 - val_accuracy: 0.8584 - val_loss: 0.5517
Epoch 7/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 80s 638ms/step - accuracy: 0.8429 - loss: 0.4803 - val_accuracy: 0.8555 - val_loss: 0.5636
Epoch 8/10
126/126 ━━━━━━━━━━━━━━━━━━━━ 82s 652ms/step - accuracy: 0.8610 - loss: 0.4597 - va

In [9]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tabulate import tabulate
import json
import random

# 📦 Install tabulate for clean table output
!pip install tabulate

# 🏷 Set paths
model_path = "/content/mobilenetv2_finetuned.h5"
class_labels_path = '/content/drive/MyDrive/emoji-detector-ai/models/class_labels.json'  # Update this path if needed
dataset_dir = "/content/classified_emojis/classified_emojis"  # Path to the classified_emojis directory

# 📊 Load class labels
with open(class_labels_path, 'r', encoding='utf-8') as f:
    class_labels = json.load(f)
class_names = list(class_labels.keys())
index_to_class = {v: k for k, v in class_labels.items()}
print(f"Loaded {len(class_names)} classes: {class_names}")

# 🧠 Load the trained model
print(f"Loading model from: {model_path}")
model = load_model(model_path)

# 📂 Collect all image paths from the classified_emojis directory
image_paths = []
true_labels = []
for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if not os.path.isdir(category_path):
        continue
    for img_name in os.listdir(category_path):
        if img_name.endswith('.png') or img_name.endswith('.jpg'):  # Include both .png and .jpg formats
            img_path = os.path.join(category_path, img_name)
            image_paths.append(img_path)
            true_labels.append(category)

# 🧩 Debugging step: Check the number of images collected
print(f"Total images found: {len(image_paths)}")
if len(image_paths) == 0:
    print("No images found. Please check the dataset path or format of images.")
else:
    # 📌 Randomly select 100 images for prediction
    num_images_to_predict = 100  # Adjust as needed
    if len(image_paths) < num_images_to_predict:
        num_images_to_predict = len(image_paths)
        print(f"Dataset has only {num_images_to_predict} images. Predicting on all available images.")
    else:
        print(f"Selecting {num_images_to_predict} random images for prediction...")

    random_indices = random.sample(range(len(image_paths)), num_images_to_predict)
    selected_image_paths = [image_paths[i] for i in random_indices]
    selected_true_labels = [true_labels[i] for i in random_indices]

    # 📈 Predict categories and track results
    print("\nPredictions:\n")
    table_data = []
    correct_predictions = 0
    predicted_labels = []
    for img_path, true_label in zip(selected_image_paths, selected_true_labels):
        # Load and preprocess the image
        img = load_img(img_path, target_size=(128, 128))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)

        # Predict
        predictions = model.predict(img_array, verbose=0)
        predicted_class_idx = np.argmax(predictions, axis=1)[0]
        predicted_class = index_to_class[predicted_class_idx]
        confidence = predictions[0][predicted_class_idx] * 100

        # Track results
        img_name = os.path.basename(img_path)
        predicted_labels.append(predicted_class)
        if predicted_class == true_label:
            correct_predictions += 1
        table_data.append([img_name, true_label, predicted_class, f"{confidence:.2f}%"])

    # Print predictions in a table
    headers = ["Image", "True Category", "Predicted Category", "Confidence"]
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

    # 📊 Calculate and display metrics
    accuracy = (correct_predictions / num_images_to_predict) * 100

    print("\n📊 Evaluation Metrics:")
    print(f"Total Images Tested: {num_images_to_predict}")
    print(f"Correct Predictions: {correct_predictions}")
    print(f"Accuracy: {accuracy:.2f}%")

    # Classification Report with all class names
    print("\nClassification Report:")
    # Ensure all class names are included, even if not present in the sample
    report = classification_report(
        selected_true_labels,
        predicted_labels,
        labels=class_names,  # Specify all class names
        target_names=class_names,
        zero_division=0  # Avoid division by zero for classes with no predictions
    )
    print(report)

    # Confusion Matrix with all class names
    print("\nConfusion Matrix:")
    cm = confusion_matrix(
        selected_true_labels,
        predicted_labels,
        labels=class_names  # Specify all class names
    )
    cm_table = [[class_names[i]] + list(cm[i]) for i in range(len(class_names))]
    headers_cm = [""] + class_names
    print(tabulate(cm_table, headers=headers_cm, tablefmt="grid"))


Loaded 28 classes: ['Animals', 'Art & Creativity', 'Birds & Insects', 'Body Parts', 'Buildings & Places', 'Celebration & Events', 'Emoji Keycaps & Digits', 'Expressions & Faces', 'Family & Relationships', 'Flags', 'Food & Drinks', 'Hand Gestures', 'Kitchen & Household', 'Love & Emotions', 'Mail & Communication Symbols', 'Medical & Health', 'Modifiers Gondi & Extras', 'Money & Finance', 'Music & Instruments', 'Nature & Plants', 'People & Body', 'Signs & Symbols', 'Sports & Games', 'Technology & Devices', 'Time & Clocks', 'Transport & Travel', 'Weather & Sky', 'Zodiac & Astrology']
Loading model from: /content/mobilenetv2_finetuned.h5


Total images found: 5042
Selecting 100 random images for prediction...

Predictions:

+--------------------------------------------------------------------------------+------------------------+------------------------+--------------+
| Image                                                                          | True Category          | Predicted Category     | Confidence   |
+================================================================================+========================+========================+==============+
| U+30_U+20E3.png                                                                | Emoji Keycaps & Digits | Emoji Keycaps & Digits | 99.97%       |
+--------------------------------------------------------------------------------+------------------------+------------------------+--------------+
| U+1F9D1_U+200D_U+1F9B3.png                                                     | People & Body          | People & Body          | 87.42%       |
+-------------------------